# <b> 0. IMPORTING LIBRARIES

In [1]:
import numpy as np
import os
from matplotlib import pyplot as plt
import tensorflow as tf
# from tqdm import tqdm
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, InputLayer, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.regularizers import l2

# <b> 1. CREATING PATHS TO DIRECTORIES

In [2]:
#files path in a dictionary
paths = dict()
# Linux pc
paths['main'] = '/home/javi/Desktop/Python/MNIST_OD'
paths['dataset'] = os.path.join(paths['main'], 'MNIST_dataset')
paths['train_data'] = os.path.join(paths['dataset'], 'train')
paths['test_data'] = os.path.join(paths['dataset'], 'test')
# Windows pc
# paths['main'] = os.path.normcase('D:\Javi\Python\MNIST_OD')#.replace('\\','/'))
# paths['dataset'] = os.path.join(paths['main'], 'MNIST_dataset'))
# paths['train_data'] = os.path.join(paths['dataset'], 'train')
# paths['test_data'] = os.path.join(paths['dataset'], 'test')
# paths['test_img'] = os.path.join(paths['test_data'], 'test')

# <b> 2. CREATING THE YOLO LIKE MODEL

## 2.1 Neuronal network model

In [8]:
class Yolo_Reshape(tf.keras.layers.Layer):
    def __init__(self, grid_size=7, no_bnb=2, no_class=10):
        super(Yolo_Reshape, self).__init__()
        self.grid_size = grid_size
        self.no_of_bnb = no_bnb
        self.no_class = no_class

    # def get_config(self):
    #     config = super().get_config().copy()
    #     config.update({
    #         'target_shape': self.target_shape
    #     })
    #     return config

    def call(self, inputs):
        # grids 7x7
        S = [self.grid_size, self.grid_size]
        # classes
        C = self.no_class
        # no of bounding boxes per grid
        B = self.no_of_bnb

        idx1 = S[0] * S[1] * C
        idx2 = idx1 + S[0] * S[1] * B

        # class probabilities
        class_probs = tf.reshape(inputs[:, :idx1], (tf.shape(inputs)[0],) + tuple([S[0], S[1], C]))
        class_probs = tf.nn.softmax(class_probs)

        #confidence
        confs = tf.reshape(inputs[:, idx1:idx2], (tf.shape(inputs)[0],) + tuple([S[0], S[1], B]))
        confs = tf.math.sigmoid(confs)

        # boxes
        boxes = tf.reshape(inputs[:, idx2:], (tf.shape(inputs)[0],) + tuple([S[0], S[1], B * 4]))
        boxes = tf.math.sigmoid(boxes)

        outputs = tf.concat([class_probs, confs, boxes], axis=3)
        return outputs

In [7]:
class MNIST_YOLO(tf.keras.Model):
    def __init__(self, inpshape, bocks_info,
                 grid_size=7, no_of_bnb=2, no_class=10):
        super().__init__()
        self.nConvs = 0
        self.nBlocks = 0
        self.grid_size = grid_size
        self.no_of_bnb = no_of_bnb
        self.no_class = no_class
        self.bocks_info = bocks_info
        self.inpshape = inpshape
        self.layersList = list()
        for key, val in self.bocks_info.items():
            self.block_CN_gen(val, self.layersList)
        self.flatten = Flatten()
        self.dense_1 = Dense((self.no_class+self.no_of_bnb*5)*self.grid_size**2, activation='sigmoid')
        # self.reshape = Reshape((self.grid_size, self.grid_size, self.no_class+self.no_of_bnb*5))
        self.reshape = Yolo_Reshape(self.grid_size, self.no_of_bnb, self.no_class)

    def block_CN_gen(self, dictionary_info, layers_list):
        '''generate a block of convolutions
        dictionary_info : dict of list {kernel_list, stride_list, padding}'''
        for key, (filt, ker, stride, pad, act) in dictionary_info.items():
            layer_name = 'block{}_conv{}'.format(self.nBlocks, self.nConvs)
            layers_list.append(Conv2D(filt, kernel_size=ker, strides=stride, padding=pad,
                               activation=act, name=layer_name))
            self.nConvs += 1
        layers_list.append(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same',
                           name='Mpooling_block{}'.format(self.nBlocks)))
        self.nBlocks += 1

    def call(self, inputs):
        # inputs = self.InputLayer
        # x = self.A(inputs)#Input(shape=self.intshape)
        x = inputs
        for layer in self.layersList:
            x = layer(x)
        x = self.flatten(x)
        x = self.dense_1(x)
        x = self.reshape(x)
        return x#Model(inputs,x)
    def summary(self):
        inp = Input(shape=self.inpshape, name="input_layer")
        model = Model(inputs=[inp], outputs=self.call(inp))
        model.summary()
        del inp, model
        # return model.summary()
            
blocks_dict = {'block1':{'b1':[32, (7,7), (1,1), 'same', 'relu']},
               'block2':{'b1':[64, (3,3), (1,1), 'same', 'relu']},
               'block3':{'b1':[64, (1,1), (1,1), 'same', 'relu'], 'b2':[128, (3,3), (1,1), 'same', 'relu'],
                         'b3':[128, (1,1), (1,1), 'same', 'relu'], 'b4':[256, (3,3), (1,1), 'same', 'relu']}}
model = MNIST_YOLO(inpshape = (144,144,1),bocks_info=blocks_dict, grid_size=7, no_of_bnb=2, no_class=10)

print(model(tf.zeros([5,144,144,1])).shape)
model.summary()
print(model.predict(tf.zeros([5,144,144,1])).shape)

# inputs = Input(shape=(140,140,1))
# x = model(inputs)
# # print(x.numpy())
# output = Yolo_Reshape(grid_size=7, no_bnb=2, no_class=10)(x)

# mymodel = Model(inputs, output)
# mymodel.summary()
# class MyModel(tf.keras.Model):

#   def __init__(self):
#     super().__init__()
#     self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
#     self.dense2 = tf.keras.layers.Dense(5, activation=tf.nn.softmax)

#   def call(self, inputs):
#     x = self.dense1(inputs)
#     return self.dense2(x)

# model = MyModel()

ResourceExhaustedError: Exception encountered when calling layer "mnist_yolo_3" (type MNIST_YOLO).

failed to allocate memory [Op:AddV2]

Call arguments received:
  • inputs=tf.Tensor(shape=(5, 144, 144, 1), dtype=float32)

In [ ]:
def block_CN_gen(dictionary_info, Input_value):
    nConvs = 0
    nBlocks = 0
    '''generate a block of convolutions
    dictionary_info : dict of list {kernel_list, stride_list, padding}'''
    x = Input_value
    for key, (filt, ker, stride, pad, act) in dictionary_info.items():
        layer_name = 'block{}_conv{}'.format(nBlocks,nConvs)
        x = Conv2D(filt, kernel_size=ker, strides=stride, padding=pad,
                   activation=act, name=layer_name)(x)
        nConvs += 1
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same',
                     name='Mpooling_block{}'.format(nBlocks))(x)
    nBlocks += 1
    return x
A = Input(shape=(140,140,1))
blocks_dict = {'b1':[10, (3,3), (1,1), 'same', 'relu'],'b2':[5, (5,5), (1,1), 'same', 'relu'],
               'b3':[2, (7,7), (1,1), 'same', 'relu']}
B = block_CN_gen(blocks_dict,A)
model = Model(A,B)
model.summary()

## 2.2 Creating a custom loss function

In [ ]:
%config Completer.use_jedi = False

In [ ]:
# A = Input(shape=(140,140,1))
blocks_dict = {'b1':[10, (3,3), (1,1), 'same', 'relu'],'b2':[5, (5,5), (1,1), 'same', 'relu'],
               'b3':[2, (7,7), (1,1), 'same', 'relu']}
A = Conv2D(5, kernel_size=(3,3), strides=(1,1), padding='same',
                   activation='relu',input_shape=(140,140,1))(tf.zeros([1,140,140,1]))
B = Conv2D(3, kernel_size=(3,3), strides=(1,1), padding='same',
                   activation='relu')(A)
model = Model(A,B)
model.summary()


In [ ]:
class MyModel(tf.keras.Model):

  def __init__(self):
    # super(MyModel, self).__init__(name='')
    super().__init__()
    self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(5, activation=tf.nn.softmax)
    self.dropout = tf.keras.layers.Dropout(0.5)

  def call(self, inputs, training=False):
    x = self.dense1(inputs)
    if training:
      x = self.dropout(x, training=training)
    return self.dense2(x)

model = MyModel()
model(tf.zeros([1, 3]))

In [ ]:
model.summary()

In [ ]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()
    self.A = Dense(32)
    self.flat = Flatten()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    # x += input_tensor
    x = (x)
    x = self.A(x)
    return tf.nn.relu(x)


block = ResnetIdentityBlock(1, [1, 2, 3])
_ = block(tf.zeros([1, 2, 3, 3]))

In [ ]:
block.summary()
block.predict(tf.zeros([1,140,140,3]))

In [ ]:
# def pascal_voc_to_dict(filepath):
#     """Function to get all the objects from the annotation XML file."""
#     with tf.io.gfile.GFile(annon_filepath, "r") as f:
#         root = xml.etree.ElementTree.parse(f).getroot()

#         # Disable pytype to avoid attribute-error due to find returning
#         # Optional[Element]
#         # pytype: disable=attribute-error
#         size = root.find("size")
#         width = float(size.find("width").text)
#         height = float(size.find("height").text)
#         filePath = size.find("filename").text

#         for obj in root.findall("object"):
#             # Get object's label name.
#             label = obj.find("name").text.lower()
#             # Get objects' pose name.
#             pose = obj.find("pose").text.lower()
#             is_truncated = obj.find("truncated").text == "1"
#             is_difficult = obj.find("difficult").text == "1"
#             bndbox = obj.find("bndbox")
#             xmax = float(bndbox.find("xmax").text)
#             xmin = float(bndbox.find("xmin").text)
#             ymax = float(bndbox.find("ymax").text)
#             ymin = float(bndbox.find("ymin").text)
#             yield {
#                       "label": label,
#                       # "pose": pose,
#                       "bbox": tfds.features.BBox(
#                           ymin / height, xmin / width, ymax / height, xmax / width
#                       ),
#                       # "is_truncated": is_truncated,
#                       # "is_difficult": is_difficult,
#                     }

In [ ]:
# # files = os.listdir(paths['train_data'])
# def map_images_from_path(path):
#     img = tf.io.read_file(path)
#     img = tf.io.decode_jpeg(img, channels=1)
#     return img

# data = tf.data.Dataset.list_files(paths['train_data']+'/*.jpg')
# data = data.map(map_images_from_path)


In [ ]:
file_path = data.as_numpy_iterator().next()
#img = tf.io.read_file(file_path)

In [ ]:
def process_path(file_path):
    label = get_label(file_path)
    # Load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [ ]:
for i in data.take(1):
    print(i.numpy())

In [ ]:
img = tf.io.read_file(file_path)
plt.imshow(tf.io.decode_jpeg(img, channels=1).numpy(),cmap='gray')

In [ ]:
tf.keras.datasets.features.BBox(1,2,3,4)

In [ ]:
import tensorflow_datasets as tfds